In [1]:
# Server
import traceback
from flask import Flask
from flask import request
from PIL import Image
from tensorflow.keras.applications.resnet50 import ResNet50
from skimage.feature import local_binary_pattern
import cv2
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np
from joblib import load


2023-03-22 15:41:49.699450: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 15:41:49.846136: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-22 15:41:49.846159: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-22 15:41:50.688979: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [ ]:
#server
import traceback
from flask import Flask, request, send_file
import PIL
from PIL import Image
model = load('/home/nouman/Downloads/svm_model.joblib')

def solution(img_path):
    # Preprocess the image
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = preprocess_input(img)
        resnet_model = ResNet50(include_top=False, weights='imagenet', pooling='avg')
        resnet_features = resnet_model.predict(np.expand_dims(img, axis=0))[0]
        image = cv2.imread(img_path, 0)
        lbp = local_binary_pattern(image, 8, 1)
        (hist, _) = np.histogram(lbp.ravel(),
                                  bins=np.arange(0, 10),
                                  range=(0, 10))
        # normalize the histogram
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-7)
        img_height, img_width, _ = img.shape
        features = np.concatenate([resnet_features, [img_height, img_width], hist])
        features= features.reshape(-1,features.shape[0])



        # Make a prediction on the image
        pred = model.predict(features)

        # Get the predicted class label
        class_idx = pred[0]
        if class_idx == 0:
            class_label = 'Boot'
        elif class_idx == 1:
            class_label = 'Sandal'
        elif class_idx == 2:
            class_label = 'Shoe'

        # Print the predicted class label
        return class_label

app = Flask(__name__)
@app.route('/test', methods=['POST'])
def test():
    try:
        img_path = request.form['img_path']
        return solution(img_path)
    except Exception as e:
        return str(traceback.format_exc())

app.run(host='0.0.0.0', port=5050)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/home/nouman/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator SVC from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
 * Running on all addresses.
 * Running on http://172.16.10.146:5050/ (Press CTRL+C to quit)
2023-03-22 15:42:00.162710: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-22 15:42:00.162740: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-22 15:42:00.162770: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on thi

1/1 [==============================] - 1s 1s/step


127.0.0.1 - - [22/Mar/2023 15:42:03] "POST /test HTTP/1.1" 200 -
